In [1]:
from itertools import combinations
import numpy as np
import icecream as ic

## Simple Test Problem

In [17]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('./lab2/test_problem.npy')

## Common tests

In [18]:
problem = np.load('lab2/problem_r2_100.npy')

In [19]:
# Negative values?
np.any(problem < 0)

np.True_

In [20]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [21]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [22]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False

In [2]:
def cost(adj, sol):

    s = np.asarray(sol, dtype=int)
    to_idx = np.roll(s, -1)
    return float(np.sum(adj[s, to_idx]))

In [3]:
def starting_point(problem):
    n = problem.shape[0]
    rng = np.random.default_rng()
    return rng.permutation(np.arange(0, n))

In [4]:
def greedy_starting_point(problem):
    """
    Generate a greedy solution using nearest neighbor heuristic.
    Start from a random city and always move to the nearest unvisited city.
    """
    n = problem.shape[0]
    rng = np.random.default_rng()
    
    # Start from a random city
    start = rng.integers(0, n)
    visited = [start]
    unvisited = set(range(n)) - {start}
    
    current = start
    while unvisited:
        # Find nearest unvisited city
        nearest = min(unvisited, key=lambda x: problem[current, x])
        visited.append(nearest)
        unvisited.remove(nearest)
        current = nearest
    
    return np.array(visited)

In [5]:
def swap_mutation(solution, mutation_rate=0.05):
    """
    swap mutation with probability: mutation_rate
    """
    new_solution = solution.copy()
    rng = np.random.default_rng()
    
    if rng.random() < mutation_rate:
        n = len(new_solution)
        idx1, idx2 = rng.integers(0, n, 2)
        while idx1 == idx2:
            idx2 = rng.integers(0, n)
            
        new_solution[idx1], new_solution[idx2] = new_solution[idx2], new_solution[idx1]
        
    return new_solution

In [6]:
def order_crossover_ox1(parent1, parent2):
    """
    Crossover OX1
    mantain certain parent1 gene and fill the splution with the remanent parent2 gene in order
    """
    rng = np.random.default_rng()
    n = len(parent1)
    
    # 1. Choose two cut points
    cut1, cut2 = rng.integers(0, n, 2)
    if cut1 > cut2:
        cut1, cut2 = cut2, cut1
    if cut1 == cut2:
        cut2 += 1 # minimun 1 elemente
        
    child = -np.ones(n, dtype=int)
    
    # 2.copy of parent1 genes
    child[cut1:cut2] = parent1[cut1:cut2]
    
    # 3. vector for parent2 genes
    parent2_genes = []
    
    genes_in_child = set(child[cut1:cut2])
    for gene in parent2:
        if gene not in genes_in_child:
            parent2_genes.append(gene)
            
    # 4. fill child with parent2 gene only if parent1 gene is not present
    child_idx = cut2
    parent2_idx = 0
    
    while parent2_idx < len(parent2_genes):
        if child_idx == n:
            child_idx = 0
            
        if child_idx == cut1:
            child_idx = cut2
        
        child[child_idx] = parent2_genes[parent2_idx]
        child_idx += 1
        parent2_idx += 1
        
    return child

In [1]:
def tournament_selection(population, costs, k=3):
    """
    choose random k element in population and return the best one
    """
    rng = np.random.default_rng()
    
    indices = rng.choice(len(population), k, replace=False)
    
    best_cost = np.inf
    best_index = -1
    
    for idx in indices:
        if costs[idx] < best_cost:
            best_cost = costs[idx]
            best_index = idx
            
    return population[best_index]

In [ ]:
def evolutionary_algorithm(problem, population_size, num_generations, 
                           tournament_size=3, mutation_rate=0.05, elitism=1, 
                           greedy_ratio=0.2, dynamic_params=True, verbose=True):
    """
    Evolutionary Algorithm for TSP with:
    - Greedy initialization: a portion of the initial population uses greedy heuristic
    - Dynamic parameter control: mutation_rate and tournament_size adapt over generations
      * Early generations: high mutation (exploration), low tournament size
      * Later generations: low mutation (exploitation), high tournament size
    """
    
    # === 1. Starting with mixed population (greedy + random) ===
    if verbose:
        print(f"Start EA: Population={population_size}, Generations={num_generations}")
        print(f"Dynamic params: {dynamic_params}, Greedy ratio: {greedy_ratio}")
    
    # Number of greedy solutions in initial population
    num_greedy = int(population_size * greedy_ratio)
    
    population = []
    # Add greedy solutions
    for _ in range(num_greedy):
        population.append(greedy_starting_point(problem))
    # Fill rest with random solutions
    for _ in range(population_size - num_greedy):
        population.append(starting_point(problem))
    
    costs = [cost(problem, ind) for ind in population]
    
    best_global_sol = population[np.argmin(costs)]
    best_global_cost = np.min(costs)

    if verbose:
        print(f"Generation 0: cost = {best_global_cost:.2f}")

    # Dynamic parameter settings
    if dynamic_params:
        # Mutation: starts high (exploration), ends low (exploitation)
        mutation_start = 0.3
        mutation_end = 0.02
        # Tournament: starts low (diversity), ends high (exploitation)
        tournament_start = max(2, tournament_size // 2)
        tournament_end = min(population_size // 2, tournament_size * 2)
    else:
        mutation_start = mutation_end = mutation_rate
        tournament_start = tournament_end = tournament_size

    # === 2. EA with dynamic parameters ===
    for gen in range(1, num_generations + 1):
        # Calculate progress ratio (0 to 1)
        progress = gen / num_generations
        
        # Dynamic mutation rate: linear decay
        current_mutation = mutation_start + (mutation_end - mutation_start) * progress
        
        # Dynamic tournament size: linear increase
        current_tournament = int(tournament_start + (tournament_end - tournament_start) * progress)
        current_tournament = max(2, current_tournament)  # Ensure at least 2
        
        new_population = []
        
        # Strong Elitism: copy elites directly without modification
        # This ensures monotonic improvement of the best fitness
        sorted_indices = np.argsort(costs)
        for i in range(elitism):
            elite_index = sorted_indices[i]
            new_population.append(population[elite_index].copy())  # Copy to prevent mutation
            
        # --- New generation ---
        while len(new_population) < population_size:
            # a. Parents selection with dynamic tournament size
            parent1 = tournament_selection(population, costs, k=current_tournament)
            parent2 = tournament_selection(population, costs, k=current_tournament)
            
            # b. Crossover
            offspring = order_crossover_ox1(parent1, parent2)
            
            # c. Mutation with dynamic rate
            offspring = swap_mutation(offspring, mutation_rate=current_mutation)
            
            new_population.append(offspring)
            
        population = new_population
        costs = [cost(problem, ind) for ind in population]
        
        # Update best global value
        current_best_cost = np.min(costs)
        if current_best_cost < best_global_cost:
            best_global_cost = current_best_cost
            best_global_sol = population[np.argmin(costs)]
            
        if gen % 100 == 0 or gen == num_generations:
            if verbose:
                print(f"Generation {gen}: cost = {best_global_cost:.2f} (mut={current_mutation:.3f}, tourn={current_tournament})")

    if verbose:
        print("End EA.")
    return best_global_sol, best_global_cost

In [11]:
import os
import glob

def run_single_experiment(filepath, pop_size=100, generations=1000,
                          mutation_rate=0.1, tournament_k=20, elitism_count=5,
                          greedy_ratio=0.2, dynamic_params=True, verbose=True):
    """
    Run a single TSP experiment with the evolutionary algorithm.
    
    New parameters (peer review suggestions):
    - greedy_ratio: fraction of initial population using greedy heuristic
    - dynamic_params: enable adaptive mutation rate and tournament size
    """
    filename = os.path.basename(filepath)
    
    if verbose:
        print(f"Processing: {filename}...")

    try:
        problem = np.load(filepath)

        # Execution EA with new parameters
        _, best_ea_cost = evolutionary_algorithm(
            problem,
            population_size=pop_size,
            num_generations=generations,
            tournament_size=tournament_k,
            mutation_rate=mutation_rate,
            elitism=elitism_count,
            greedy_ratio=greedy_ratio,
            dynamic_params=dynamic_params,
            verbose=verbose
        )

        random_sol = starting_point(problem)
        random_cost = cost(problem, random_sol)

        if verbose:
            print(f" -> Done. EA: {best_ea_cost:.2f} | Random: {random_cost:.2f}")

        return {
            'filename': filename,
            'ea_cost': best_ea_cost,
            'random_cost': random_cost,
            'error': None
        }

    except Exception as e:
        if verbose:
            print(f" -> ERROR: {e}")
        return {
            'filename': filename,
            'ea_cost': None,
            'random_cost': None,
            'error': str(e)
        }

In [ ]:
input_folder='lab2'
output_file='result.txt'
problem_files = sorted(glob.glob(os.path.join(input_folder, '*.npy')))
if not problem_files:
    print(f"Error")

with open(output_file, 'w') as f:
    f.write(f"{'File Name':<30} | {'EA Cost':<15} | {'Random Cost':<15}\n")
    f.write("-" * 65 + "\n")

    for filepath in problem_files:
        print(f"Processing: {filepath}...")
        res = run_single_experiment(filepath, verbose=False)

        if res['error'] is None:
            line = f"{res['filename']:<30} | {res['ea_cost']:<15.2f} | {res['random_cost']:<15.2f}\n"
        else:
            line = f"{res['filename']:<30} | {'ERROR':<15} | {res['error']}\n"

        f.write(line)
        f.flush()

    print(f"\nBatch completed.")

In [12]:

risultato = run_single_experiment('lab2/problem_g_100.npy')

Processing: problem_g_100.npy...
Start EA: Population=100, Generations=1000
Dynamic params: True, Greedy ratio: 0.2
Generation 0: cost = 4495.76
Generation 100: cost = 4404.70 (mut=0.272, tourn=13)
Generation 200: cost = 4404.70 (mut=0.244, tourn=16)
Generation 300: cost = 4404.70 (mut=0.216, tourn=19)
Generation 400: cost = 4404.70 (mut=0.188, tourn=22)
Generation 500: cost = 4404.70 (mut=0.160, tourn=25)
Generation 600: cost = 4404.70 (mut=0.132, tourn=28)
Generation 700: cost = 4404.70 (mut=0.104, tourn=31)
Generation 800: cost = 4404.70 (mut=0.076, tourn=34)
Generation 900: cost = 4404.70 (mut=0.048, tourn=37)
Generation 1000: cost = 4404.70 (mut=0.020, tourn=40)
End EA.
 -> Done. EA: 4404.70 | Random: 25629.83


In [13]:
# Quick test on g_500 to verify improvements on larger problems
risultato_500 = run_single_experiment('lab2/problem_g_500.npy', generations=500, verbose=True)

Processing: problem_g_500.npy...
Start EA: Population=100, Generations=500
Dynamic params: True, Greedy ratio: 0.2
Generation 0: cost = 9715.85
Generation 100: cost = 9698.65 (mut=0.244, tourn=16)
Generation 200: cost = 9683.27 (mut=0.188, tourn=22)
Generation 300: cost = 9683.27 (mut=0.132, tourn=28)
Generation 400: cost = 9683.27 (mut=0.076, tourn=34)
Generation 500: cost = 9683.27 (mut=0.020, tourn=40)
End EA.
 -> Done. EA: 9683.27 | Random: 130504.79
